## Installing necessary packages

In [ ]:
!pip install transformers --q

## Importing necessary packages

In [ ]:
import pandas as pd
import numpy as np
from numpy import zeros

from tqdm import tqdm

import json
import torch

from transformers import AutoModel, AutoTokenizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score


import torch as th
from transformers import AutoModel, AutoTokenizer
import torch.utils.data as Data
from torch.optim import lr_scheduler
import torch.nn.functional as F

import matplotlib.pyplot as plt
%matplotlib inline

## Parameters

In [ ]:
sequence_length = 256
embedding_dim = 768
batch_size = 32
epochs = 6
model_name = 'nlpaueb/legal-bert-base-uncased'
output_type = 'mean_pooled' # by default CLS

## Dataset Loader

#### Training dataset

In [ ]:
train_df = pd.read_csv('../../data/model_data/train_df.csv')
print("# Size: ", train_df.shape)
train_df.head()

In [ ]:
print("#Nan in sents: ", train_df.sentence.isnull().values.sum())
print("#Nan in labels: ", train_df.label.isnull().values.sum())

In [ ]:
train_df.label.value_counts()

#### Validation dataset

In [ ]:
dev_df = pd.read_csv('../../data/model_data/dev_df.csv')
print("# Size: ", dev_df.shape)
dev_df.head()

In [ ]:
print("#Nan in sents: ", dev_df.sentence.isnull().values.sum())
print("#Nan in labels: ", dev_df.label.isnull().values.sum())

In [ ]:
dev_df.label.value_counts()

#### Reading labels

In [ ]:
le_encoder = LabelEncoder()
le_encoder.fit(list(train_df.label.values) + list(dev_df.label.values))

label_names = le_encoder.classes_
print(label_names)

In [ ]:
ID_Label_Map = {}
with open('../../data/model_data/ID_Label_Map.json', 'r') as fp:
    ID_Label_Map = json.load(fp)

Label_ID_Map = {}
with open('../../data/model_data/Label_ID_Map.json', 'r') as fp:
    Label_ID_Map = json.load(fp)

print(list(Label_ID_Map.keys()))

In [ ]:
train_labels = le_encoder.transform(list(train_df.label.values))
print("# Labels: ", len(train_labels))

In [ ]:
dev_labels = le_encoder.transform(list(dev_df.label.values))
print("# Labels: ", len(dev_labels))

## Bert Model

In [ ]:
class DenseLayer(th.nn.Module):
    def __init__(self, bert_hidden_size, num_labels):
        super(DenseLayer, self).__init__()
        self.linear1 = th.nn.Linear(bert_hidden_size, 512)
        self.linear2 = th.nn.Linear(512, 256)
        self.linear3 = th.nn.Linear(256, num_labels)
        self.dropout = th.nn.Dropout(0.1)
        self.relu = th.nn.ReLU()
        
    def forward(self, x):
        x = self.linear1(x)
        x = self.dropout(x)
        x = self.relu(x)
        x = self.linear2(x)
        x = self.dropout(x)
        x = self.relu(x)
        x = self.linear3(x)
        return x

In [ ]:
class BertClassifier(th.nn.Module):
    def __init__(self, pretrained_model='roberta-base', nb_class=20):
        super(BertClassifier, self).__init__()
        self.nb_class = nb_class
        self.tokenizer = AutoTokenizer.from_pretrained(pretrained_model)
        self.bert_model = AutoModel.from_pretrained(pretrained_model)
        self.feat_dim = list(self.bert_model.modules())[-2].out_features
        self.classifier = DenseLayer(self.feat_dim, nb_class)

    def forward(self, input_ids, attention_mask, output_type = 'CLS'):

        cls_feats = None
        if output_type=='mean_pooled':
            bert_output = self.bert_model(input_ids, attention_mask).last_hidden_state
            
            ### Mean Pooling
            cls_feats = bert_output.sum(axis=1) / attention_mask.sum(axis=-1).unsqueeze(-1)
        else:
            cls_feats = self.bert_model(input_ids, attention_mask)[0][:, 0]
        
        logits = self.classifier(cls_feats)
        
        return logits

In [ ]:
model = BertClassifier(pretrained_model = model_name, nb_class=len(ID_Label_Map))

#### Preparing the data loader for bert classifier

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def encode_input(text, tokenizer):
    tokenized_text = tokenizer(
        text,
        max_length = sequence_length,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    return tokenized_text.input_ids, tokenized_text.attention_mask

In [ ]:
input_ids, attention_mask = {}, {}

In [ ]:
input_ids['train'], attention_mask['train'] = encode_input(list(train_df.sentence.values), model.tokenizer)

print(input_ids['train'].shape, type(input_ids['train']))
print(attention_mask['train'].shape, type(attention_mask['train']))

In [ ]:
dev_sentences = list(dev_df.sentence.values)
dev_actual_labels = list(dev_df.label.values)

In [ ]:
input_ids['val'], attention_mask['val'] = encode_input(dev_sentences, model.tokenizer)

print(input_ids['val'].shape, type(input_ids['val']))
print(attention_mask['val'].shape, type(attention_mask['val']))

In [ ]:
label_encode = {}
label_encode['train'] = th.LongTensor(train_labels)
label_encode['val'] = th.LongTensor(dev_labels)

print(label_encode['train'].shape, type(label_encode['train']))
print(label_encode['val'].shape, type(label_encode['val']))

In [ ]:
datasets = {}
loader = {}
for split in ['train', 'val']:
    datasets[split] =  Data.TensorDataset(input_ids[split], attention_mask[split], label_encode[split])
    split_sampler = None
    if split == 'train':
        split_sampler = Data.RandomSampler(datasets[split])
    else:
        split_sampler = Data.SequentialSampler(datasets[split])  
    loader[split] = Data.DataLoader(datasets[split], sampler = split_sampler, batch_size = batch_size)

In [ ]:
print("#train Dataset: ", len(datasets['train']))
print("#train Labels: ", len(label_encode['train']))
print("#train Loader: ", len(loader['train']))

print("#dev Dataset: ", len(datasets['val']))
print("#dev Labels: ", len(label_encode['val']))
print("#dev Loader: ", len(loader['val']))

#### Optimizer and scheduler

In [ ]:
optimizer = th.optim.Adam(model.parameters(), lr=1e-4)
scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[30], gamma=0.1)

#### Training

In [ ]:
train_loss = []
val_loss = []

train_acc = []
val_acc = []

best_val_loss = None

for epoch in range(epochs):
    print("Epoch: {}/{}".format(epoch + 1 , epochs))
    
    t_loss, t_acc = 0, 0
    v_loss, v_acc = 0, 0
    
    actual_labels, predictions = [], []
    
    for step, batch in enumerate(loader['train']):
        
        # progress update after every 50 batches.
        print('\rBatch {:>5,}  of  {:>5,}.'.format(step, len(loader['train'])), end='')
    
        model.train()

        model = model.to(device)
        optimizer.zero_grad()

        (input_ids, attention_mask, label) = [x.to(device) for x in batch]
        optimizer.zero_grad()

        y_pred = model(input_ids, attention_mask)
        y_true = label.type(th.long)

        loss = F.cross_entropy(y_pred, y_true)
        loss.backward()

        optimizer.step()
        t_loss += loss.item()

        with th.no_grad():
            y_true = y_true.detach().cpu()
            y_pred = y_pred.argmax(axis=1).detach().cpu()
            t_acc += accuracy_score(y_true, y_pred)
            
            for i in y_true:
                actual_labels.append(i)
                
            for i in y_pred:
                predictions.append(i)
                
    train_loss.append(t_loss/len(loader['train']))
    train_acc.append(t_acc/len(loader['train']))
    f1_measure = f1_score(actual_labels, predictions, average='micro')
    
    print("Training Loss: ", t_loss/len(loader['train']))
    print("Train Accuracy: ", t_acc/len(loader['train']))
    print("F1 measure: ", f1_measure)
    
    #### Validating the validation samples
    predictions, actual_labels = [], []
    with th.no_grad():
        for step, batch in enumerate(loader['val']):
            
            # progress update after every 50 batches.
            print('\rBatch {:>5,}  of  {:>5,}.'.format(step, len(loader['val'])), end="")
                
            model.eval()
            model = model.to(device)
            
            (input_ids, attention_mask, label) = [x.to(device) for x in batch]
            optimizer.zero_grad()
            
            y_pred = model(input_ids, attention_mask)
            y_true = label.type(th.long)
            
            loss = F.cross_entropy(y_pred, y_true)
            v_loss += loss.item()
            
            y_true = y_true.detach().cpu()
            y_pred = y_pred.argmax(axis=1).detach().cpu()
            
            for i in y_true:
                actual_labels.append(i)
                
            for i in y_pred:
                predictions.append(i)
                
            v_acc += accuracy_score(y_true, y_pred)
            
    val_loss.append(v_loss/len(loader['val']))
    val_acc.append(v_acc/len(loader['val']))
    
    f1_measure = f1_score(actual_labels, predictions, average='micro')
    
    print("Validation Loss: ", v_loss/len(loader['val']))
    print("Validation Accuracy: ", v_acc/len(loader['val']))
    print("Validation F1 measure: ", f1_measure)
    
    scheduler.step()
    
    if best_val_loss is None or best_val_loss > (v_acc/len(loader['val'])):
        th.save(
            {
                'bert_model': model.bert_model.state_dict(),
                'classifier': model.classifier.state_dict(),
                'optimizer': optimizer.state_dict(),
                'epoch': epochs,
            },
            '../../checkpoints/checkpoint.pth'
        )
        best_val_loss = (v_acc/len(loader['val']))
        
        pred_labels = []
        for la in predictions:
            pred_labels.append(ID_Label_Map.get(str(la.item()), ''))
        df = pd.DataFrame(data = {'sentence': dev_sentences, 'actual': dev_actual_labels, 'predict': pred_labels}, columns = ['sentence', 'actual', 'predict'])
        df.to_csv('../../model_results/eval_predictions.csv')

### Accuracy and Loss plots

In [ ]:
fig, (ax1, ax2) = plt.subplots(figsize = (8, 4), nrows = 1, ncols=2)

ax1.plot(range(epochs), train_loss, label='train', color='red')
ax1.scatter(range(epochs), train_loss, color='red')
ax1.plot(range(epochs), val_loss, label='dev', color='blue')
ax1.scatter(range(epochs), val_loss, color='blue')
ax1.set(xlabel = 'Epochs', ylabel = 'Loss')
ax1.set_title('Train vs Dev Loss')
ax1.legend()

ax2.plot(range(epochs), train_acc, label='train', color='red')
ax2.scatter(range(epochs), train_loss, color='red')
ax2.plot(range(epochs), val_acc, label='dev', color='blue')
ax2.scatter(range(epochs), val_acc, color='blue')
ax2.set(xlabel = 'Epochs', ylabel = 'Accuracy')
ax2.set_title('Train vs Dev Accuracy')
ax2.legend()

fig.savefig('train_vs_test_loss_and_accuracy.jpg')
fig.show()

## Test Data Inference and submission file preparation

### Load best model

In [ ]:
model = BertClassifier(pretrained_model='../../checkpoints/checkpoint.pth', nb_class=len(Label_ID_Map))

### Load the labels list

In [ ]:
print(ID_Label_Map)
labels_list = []
for i in range(len(ID_Label_Map)):
    label = ID_Label_Map[str(i)]
    labels_list.append(label)

print(labels_list)

### Predictions for the testset

In [ ]:
rr_test_data = {}
with open('../../data/model_data/SAMPLE_SUBMISSION_RR.json', 'r') as fp:
    rr_test_data = json.load(fp)
print("# Documents: ", len(rr_test_data))

In [ ]:
final_results = rr_test_data.copy()
for doc_index, entry in tqdm(enumerate(rr_test_data)):
    results = rr_test_data[doc_index]['annotations'][0]['result']
    for sent_index, sent in enumerate(results):
        
        #### Extracting the text from the test data
        sentence = sent['value']['text']
        sentence = sentence.replace(r'\s+', ' ').strip()
        
        #### Getting input ids and attention mask from the model for the given sentence
        input_ids, attention_mask = encode_input([sentence], model.tokenizer)
        
        label = None
        #### Predicting the label index from the model
        with th.no_grad():
            model.eval()
            model.to(device)
            y_pred = model(input_ids.to(device), attention_mask.to(device)).cpu()
            label = labels_list[np.argmax(y_pred)]
        
        if label is not None:
            final_results[doc_index]['annotations'][0]['result'][sent_index]['value']['labels'] = [label]
        else:
            print("Label is None for Doc-index: {} and Sent-index: {}".format(doc_index, sent_index))

print("# Documents: ", len(final_results))

In [ ]:
with open('../../output/RR_SUBMISSION.json', 'w') as fp:
    json.dump(final_results, fp, indent=4)